In [1]:
import sys
import json
import csv
import yaml

import pandas as pd
import numpy as np

import datetime;

import matplotlib as mpl

import time
from datetime import datetime

import pprint

import os
from dotenv import load_dotenv

import psycopg2
from sqlalchemy import create_engine, text as sql_text

In [2]:
dotenv_path = 'variables.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:

load_dotenv()

schema = os.getenv('DISC_4_SCHEMA')
port = os.getenv('DISC_4_PORT')
host = os.getenv('DISC_4_HOST')
database = os.getenv('DISC_4_DB')
username = os.getenv('username')
password = os.getenv('password')

In [4]:
db_eng = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}",
                       connect_args={'options': '-csearch_path={}'.format(f"{schema}")},
                       isolation_level = 'SERIALIZABLE')

print("Successfully created db engine.")

Successfully created db engine.


In [5]:
sys.path.append('/Users/Nfaith21/Documents/ECS 116 - Misc/DISC_5_FILES/benchmarking/')
import util_actual_main as util

# Step 3a - listings_join_reviews.json

In [6]:
new_dict = {}

years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
for year in years:
    new_dict['q_listings_join_reviews_'+year] = util.build_query_listings_join_reviews_datetime(year+'-01-01 12:00:00.000', year+'-12-31 12:00:00.000')


In [ ]:
all_perf_details = {}

all_indexes = [['datetime','reviews'], ['id','listings']] 


index_combinations = [
    [],
    [['datetime', 'reviews']],
    [['id', 'listings']],
    [['datetime', 'reviews'], ['id', 'listings']]
]

count = 50

for query_name, query in new_dict.items():
    print('Processing query:', query_name)

    query_perf_details = {}

    for spec in index_combinations:
        spec_name = util.build_index_description_key(all_indexes, spec) if spec else '__'

        print('Processing spec:', spec_name)

        for index in all_indexes:
            if index not in spec:
                mod_index = util.add_drop_index(db_eng, 'drop', index[0], index[1])

        for index in spec:
            mod_index = util.add_drop_index(db_eng, 'add', index[0], index[1])

        perf_details = util.calc_time_diff_per_year(db_eng, count, {query_name: query})

        query_perf_details[spec_name] = perf_details[query_name]

    all_perf_details[query_name] = query_perf_details
    print('Finished processing query:', query_name, '\n')

util.write_perf_data(all_perf_details, 'listings_join_reviews.json')


In [ ]:
  q5 = """
SELECT CAST(EXTRACT(YEAR FROM datetime) AS INTEGER) AS year, COUNT(*) AS review_count
FROM reviews
group by year;
"""
with db_eng.connect() as conn:
    result_reviews = conn.execute(sql_text(q5))
    print()
    print(result_reviews.all())


# Step 3b plots - text_search_query.json

In [ ]:
text_search_dict = {}
words = ['apartment','awesome','horrible']
years = ['2009', '2010', '2011', '2012', '2013', '2014', '2017', '2019', '2023']

for word in words:
    for year in years:
        text_search_dict[word+'_'+year] = util.text_search(db_eng, year+'-01-01 12:00:00.000',year+'-12-31 12:00:00.000',word, 50)
text_search_dict
util.write_perf_data(text_search_dict, 'text_search_query.json')

# Step 3c plots - update_datetimes_query.json

In [ ]:
# neighborhoods and indexes
neighbourhoods = ['New Springville', 'Fort Hamilton', 'Long Island City', 'Bedford-Stuyvesant']
neighbourhoods_all_indexes = [['datetime','reviews'], ['neighbourhood','listings']]

neighborhoods_q_dict = {}

for nb in neighbourhoods:
    neighborhoods_q_dict[f'update_datetimes_query_{nb}'] = (util.build_query_reviews_datetime_update(5, '+', 'neighbourhood', nb), util.build_query_reviews_datetime_update(5, '-', 'neighbourhood', nb))

In [ ]:
util.run_update_datetime_query_neighbourhoods(db_eng, 50, neighborhoods_q_dict, neighbourhoods_all_indexes, 'update_datetimes_query.json')

In [ ]:
# neighbourhood_groups and indexes
neighbourhood_groups = ['Staten Island', 'Bronx', 'Queens', 'Manhattan']
neighborhood_groups_all_indexes = [['datetime','reviews'], ['neighbourhood_group','listings']]

neighborhood_groups_q_dict = {}

for group in neighbourhood_groups:
    neighborhood_groups_q_dict[f'update_datetimes_query_{group}'] = (util.build_query_reviews_datetime_update(5, '+', 'group', group), util.build_query_reviews_datetime_update(5, '-', 'group', group))

In [ ]:
util.run_update_datetime_query_neighbourhoods(db_eng, 50, neighborhood_groups_q_dict, neighborhood_groups_all_indexes, 'update_datetimes_query.json')